In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_clean = pd.read_csv('hw4-trainingset-wsa2113.csv')
train = train_clean.copy()

/Users/barmfield/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (50,53,54,55,56,255,256,257,258,260,268,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_clean = pd.read_csv('hw4-testset-wsa2113.csv')
test = test_clean.copy()

/Users/barmfield/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268,280) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cb = pd.read_csv('CodeBook-SELECT.csv')

In [5]:
print('train shape:',train.shape)
print('test shape:',test.shape)
print('code book shape:',cb.shape)

train shape: (20000, 380)
test shape: (24500, 380)
code book shape: (379, 2)


## TODO:  
* create baseline
    * drop high null variables and rows
* Data cleaning
    * drop duplicate columns
        * cntryid, cntryid_e
        * drop cntryid, as it is less specific
    * drop cols w/ high null %
    * drop rows w/ high null %
    * add is_null column for each category where value was null
    * find cols with mixed types and correct
    * convert numerical columns to int or float from objects
    * categorical variables
        * OHC categorical variables
        * make sure dtypes match up across train and test
    * missing data
        * experiment imputing with mean and median
            * check distribution of data; if there are big outliers, use median, if not, use mean
        * check all cols which have missing data and see if that data should be captured in a new feature
    * feature scaling
        * **NOTE: split and scale at the same time**
        * normalization --> min/max scaler
        * standardization (z-score normalization)
        * api
            * column transformer
                * numeric transformer
                    * imputer
                    * scaler
                * categorical transformer
                    * imputer
                    * OHE
    * dimensionality reduction
        * PCA
    * normalizing
        * yeo-johnson power transformer

# Baseline
* Remove columns
    * Drop high % null cols and rows
        * Threshold = 75%
    * Drop columns unrelated to performance
    * Drop target


* Split data into train, val, and test sets

In [10]:
from sklearn.model_selection import train_test_split

train = train\
    .dropna(thresh = len(train)*0.25, axis = 1)\
    .dropna(thresh = len(train.columns)*0.25, axis = 0)

X_train = train.drop(['uni','row','job_performance'], axis = 1)
y_train = train['job_performance']
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 4771)
X_test = test[X_train.columns]
y_test = test['job_performance']
print(X_train.shape)
print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(19994, 273)
(19994,)
(24500, 273)
(24500,)


* Identify categorical and numerical columns

In [7]:
cat_cols = ['cntryid', 'cntryid_e','gender_r', 'computerexperience', 'nativespeaker',
            'edlevel3','monthlyincpr', 'yearlyincpr','lng_home','cnt_h','cnt_brth', 'edcat7',
            'reg_tl2','lng_bq','lng_ci','vet','ctryqual','birthrgn','nativelang','ctryrgn',
            'imyrcat','ageg5lfs','ageg10lfs','ageg10lfs_t','edcat8','edcat6','fe12','aetpop',
            'faet12','faet12jr','faet12njr','nfe12','fnfaet12','edwork','neet','nopaidworkever',
            'paidwork12','iscoskil4','isic1l','nfe12jr','fnfe12jr','fnfaet12jr','fnfaet12njr',
            'paidwork5','earnhrdcl','earnhrbonusdcl','earnmthalldcl','earnflag','learnatwork_wle_ca',
            'readytolearn_wle_ca','icthome_wle_ca','ictwork_wle_ca','influence_wle_ca',
            'planning_wle_ca','readwork_wle_ca','taskdisc_wle_ca','writhome_wle_ca','writwork_wle_ca',
            'v59','v200','v31','v151','v272','v90','v157','v74','v153','v102', 'v101', 'v60', 'v285',
            'v17', 'v94', 'v137', 'v234', 'v91', 'v269', 'v236', 'v47', 'v291',
            'v227', 'v225', 'v201', 'v36', 'v209', 'v185', 'v238', 'v35',
            'v183', 'v5', 'v273', 'v259', 'v286', 'v95', 'v67', 'v217', 'v174',
            'v205', 'v46', 'v122', 'v92', 'v88', 'v179', 'v52', 'v97', 'v84',
            'v270', 'v33', 'v242', 'v140', 'v53', 'v82', 'v70', 'v184', 'v19',
            'v104', 'v48', 'v8', 'v115', 'v251', 'v3', 'v135', 'v235', 'v1',
            'v261', 'v263', 'v158', 'v244', 'v198', 'v212', 'v62', 'v221',
            'v214', 'v276', 'v246', 'v181', 'v96', 'v63', 'v87', 'v255',
            'v289', 'v77', 'v123', 'v141', 'v24', 'v193', 'v275', 'v204',
            'v108', 'v164', 'v166', 'v197', 'v34', 'v42', 'v292', 'v131',
            'v142', 'v188', 'v139', 'v247', 'v99', 'v180', 'v124', 'v51',
            'v190', 'v248', 'v229', 'v189', 'v165', 'v173', 'v134', 'v2',
            'v25', 'v18', 'v216', 'v178', 'v282', 'v13', 'v233', 'v278',
            'v103', 'v155', 'v152', 'v258', 'v277', 'v40', 'v146', 'v195',
            'v73', 'v23', 'v106', 'v271', 'v250', 'v176', 'v111', 'v218',
            'v253', 'v132', 'v284', 'v267', 'v260', 'v26', 'v171', 'v14', 'v7',
            'v240', 'v186', 'v162', 'v149', 'v228', 'v28', 'v237', 'v280',
            'v175', 'v288', 'v15', 'v208', 'v43', 'v27', 'v114', 'v191',
            'v170', 'v65', 'v57', 'v177', 'v69', 'v85', 'v50', 'v239', 'v224',
            'v71', 'v105','isic2l','isic1c','isic2c','isco1c','isco2c','isco1l',
            'isco2l','nfe12njr','readhome_wle_ca','v256'
           ]

num_cols = ['age_r','yrsqual','yrsqual_t','yrsget','leavedu','nfehrsnjr','nfehrsjr',
            'nfehrs','earnhr','earnhrppp','earnhrbonus','earnhrbonusppp','earnmth',
            'earnmthppp','earnmthbonus','earnmthall','earnmthallppp','earnmthbonusppp','learnatwork',
            'readytolearn','icthome','ictwork','influence','planning','readhome','readwork','taskdisc',
            'writhome','writwork','v202','v231'
           ]

# categorical_obj_cols = X_train.dtypes == object

Dropped 6 rows and 105 columns.

* Fix columns with multiple data types

In [11]:
for col in cat_cols:
    X_train[col] = X_train[col].astype('str')
    X_val[col] = X_val[col].astype('str')
    
for col in num_cols:
    X_train[col] = X_train[col].astype('float')
    X_val[col] = X_val[col].astype('float')

### Build pipeline
    * impute nulls
    * scale numerical columns
    * OHE categorical columns
    * Lasso regression to remove variables

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score

num_transform = Pipeline(
    [
        ('num_imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

cat_transform = Pipeline(
    [
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    [
        ('num', num_transform, num_cols),
        ('cat', cat_transform, cat_cols)
    ]
)

clf = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('lasso', Lasso())
    ]
)

scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='neg_mean_squared_error')
print('Average MSE = {:.2f} +/- {:.2f}'.format(-np.mean(scores), np.std(scores)))

Average MSE = 126635.26 +/- 9758.76


###

In [ ]:
preprocessor.fit(X_train)
X_train_clean = preprocessor.transform(X_train)
X_val_clean = preprocessor.transform(X_val)

In [10]:
# high cardinality cols
for col in cat_cols:
    if len(train[col].unique()) > 50:
        print(col)


lng_home
cnt_h
cnt_brth
reg_tl2
v272
v52
v184
v104
v135
v235
v1
v63
v87
v239
v224
v71
v105
isic2l
isic2c
isco2c
isco2l


In [22]:
cat_transform.fit(X_train[cat_cols])

Pipeline(memory=None,
     steps=[('cat_imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='most_frequent', verbose=0)), ('onehot', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True))])

In [13]:
np.mean(scores)

NameError: name 'scores' is not defined

In [581]:
cat_transform.fit(X_train[cat_cols])

Pipeline(memory=None,
     steps=[('cat_imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='most_frequent', verbose=0)), ('onehot', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True))])

In [583]:
transformed = cat_transform.transform(X_train[cat_cols])
transformed.shape

(14995, 5375)

In [501]:
import sklearn
sklearn.show_versions()


System:
    python: 3.7.3 (default, Mar 27 2019, 16:54:48)  [Clang 4.0.1 (tags/RELEASE_401/final)]
executable: /Users/barmfield/anaconda3/bin/python
   machine: Darwin-19.0.0-x86_64-i386-64bit

BLAS:
    macros: SCIPY_MKL_H=None, HAVE_CBLAS=None
  lib_dirs: /Users/barmfield/anaconda3/lib
cblas_libs: mkl_rt, pthread

Python deps:
       pip: 18.1
setuptools: 40.6.3
   sklearn: 0.20.1
     numpy: 1.15.4
     scipy: 1.1.0
    Cython: 0.29.2
    pandas: 0.23.4


In [105]:
# find columns with mixed types

# for col in train.columns:
#     weird = (train[[col]].applymap(type) != train[[col]].iloc[0].apply(type)).any(axis=1)
#     if len(train[weird]) > 0:
#         print(col)

In [71]:
train.dtypes

Index(['isic2l', 'isco1c', 'isco2c', 'isco1l', 'isco2l', 'v76', 'v144', 'v199',
       'v44', 'v109', 'v172', 'v71', 'v10'],
      dtype='object')

In [58]:
train[train.cntryid != train.cntryid_e]

In [59]:
# train.isnull().sum().sort_values(ascending=False)/train.shape[0]> .5

In [77]:
train.shape

(19994, 275)

Drop columns greater than 75% null